In [ ]:
from llamabot import SimpleBot
from llamabot.prompt_library.diffbot import diffbot, get_github_diff

In [ ]:
url = "https://github.com/pyjanitor-devs/pyjanitor/pull/1262"


# print(get_github_diff(url))

In [ ]:
from llamabot.prompt_library.diffbot import (
    summarize,
    describe_advantages,
    suggest_improvements,
    diffbot,
)


diff = get_github_diff("https://github.com/pyjanitor-devs/pyjanitor/pull/1262")
print(diff)

In [ ]:
from llamabot.prompt_library.git import (
    commitbot,
    write_commit_message,
    compose_release_notes,
)

In [ ]:
from llamabot.prompt_manager import prompt

In [ ]:
@prompt
def structured_commit_message(diff):
    """Please write a commit message for the following diff.

    {{ diff }}

    # noqa: DAR101

    Use the Conventional Commits specification to write the diff.

    Return it for me in JSON format:

    {
        "type": "<type>",
        "scope": "<scope>",
        "description": "<description>",
        "breaking_change": "<breaking_change>",
        "body": ["<bullet point 1 no bullet>", "<bullet point 2 no bullet>",...],
        "footer": "<footer>"
    }

    The commit contains the following structural elements,
    to communicate intent to the consumers of your library:

    fix: a commit of the type fix patches a bug in your codebase
        (this correlates with PATCH in Semantic Versioning).
    feat: a commit of the type feat introduces a new feature to the codebase
        (this correlates with MINOR in Semantic Versioning).
    BREAKING CHANGE: a commit that has a footer BREAKING CHANGE:,
        or appends a ! after the type/scope,
        introduces a breaking API change
        (correlating with MAJOR in Semantic Versioning).
        A BREAKING CHANGE can be part of commits of any type.

    types other than fix: and feat: are allowed,
    for example @commitlint/config-conventional
    (based on the Angular convention) recommends
    build:, chore:, ci:, docs:, style:, refactor:, perf:, test:, and others.

    footers other than BREAKING CHANGE: <description> may be provided
    and follow a convention similar to git trailer format.

    Additional types are not mandated by the Conventional Commits specification,
    and have no implicit effect in Semantic Versioning
    (unless they include a BREAKING CHANGE).
    A scope may be provided to a commit's type,
    to provide additional contextual information and is contained within parenthesis,
    e.g., feat(parser): add ability to parse arrays.
    Within the optional body section, prefer the use of bullet points.

    Final instructions:

    1. Do not fence the commit message with back-ticks or quotation marks.
    2. Do not add any other text except the commit message itself.
    3. Only write out the commit message.
    """

In [ ]:
from llamabot import SimpleBot


bot = SimpleBot(
    "You are an expert Git user", json_mode=True,
)
response = bot(structured_commit_message(diff))

In [ ]:
import json


@prompt
def rehydrate_commit_message(commit_json):
    """{{ commit_json["type"] }}({{ commit_json["scope"] }}): {{ commit_json["description"] }}
{% for item in commit_json["body"] %}
- {{ item }}{% endfor %}

    {% if commit_json["footer"] %}
    {{ commit_json["footer"] }}
    {% endif %}
    """


print(rehydrate_commit_message(json.loads(response.content)))

In [ ]:
from llamabot import SimpleBot


bot = SimpleBot("You are an expert Git user", json_mode=True)
response = bot(structured_commit_message(diff))

In [ ]:
# print(write_commit_message(diff))

In [ ]:
bot = commitbot()
bot(write_commit_message(diff))
# diffbot(describe_advantages(diff))

In [ ]:
diffbot(suggest_improvements(diff))

In [ ]:
asdfasdfadsf